<a href="https://colab.research.google.com/github/Sushilchankapure/Sushilchankapure/blob/main/Understanding_and_coding_the_Self_Attention_Mechansim_of_Large_Language_Models_From_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
sentence = 'Life is short, eat dessert first'

dc = {s:i for i, s in enumerate(sorted(sentence.replace(',','').split()))}

print(dc)

{'Life': 0, 'dessert': 1, 'eat': 2, 'first': 3, 'is': 4, 'short': 5}


In [3]:
import torch

sentence_int = torch.tensor([dc[s] for s in sentence.replace(',','').split()])

print(sentence_int)

tensor([0, 4, 5, 2, 1, 3])


In [4]:
torch.manual_seed(123)
embed = torch.nn.Embedding(6,16)
embedded_sentence = embed(sentence_int).detach()

print(embedded_sentence)
print(embedded_sentence.shape)

tensor([[ 0.3374, -0.1778, -0.3035, -0.5880,  0.3486,  0.6603, -0.2196, -0.3792,
          0.7671, -1.1925,  0.6984, -1.4097,  0.1794,  1.8951,  0.4954,  0.2692],
        [ 0.5146,  0.9938, -0.2587, -1.0826, -0.0444,  1.6236, -2.3229,  1.0878,
          0.6716,  0.6933, -0.9487, -0.0765, -0.1526,  0.1167,  0.4403, -1.4465],
        [ 0.2553, -0.5496,  1.0042,  0.8272, -0.3948,  0.4892, -0.2168, -1.7472,
         -1.6025, -1.0764,  0.9031, -0.7218, -0.5951, -0.7112,  0.6230, -1.3729],
        [-1.3250,  0.1784, -2.1338,  1.0524, -0.3885, -0.9343, -0.4991, -1.0867,
          0.8805,  1.5542,  0.6266, -0.1755,  0.0983, -0.0935,  0.2662, -0.5850],
        [-0.0770, -1.0205, -0.1690,  0.9178,  1.5810,  1.3010,  1.2753, -0.2010,
          0.4965, -1.5723,  0.9666, -1.1481, -1.1589,  0.3255, -0.6315, -2.8400],
        [ 0.8768,  1.6221, -1.4779,  1.1331, -1.2203,  1.3139,  1.0533,  0.1388,
          2.2473, -0.8036, -0.2808,  0.7697, -0.6596, -0.7979,  0.1838,  0.2293]])
torch.Size([6, 16])


In [5]:
torch.manual_seed(123)

d = embedded_sentence.shape[1]

d_q, d_k, d_v = 24, 24, 28

W_q = torch.nn.Parameter(torch.rand(d_q, d))
W_key = torch.nn.Parameter(torch.rand(d_k, d))
W_value = torch.nn.Parameter(torch.rand(d_v, d))

In [6]:
W_q

Parameter containing:
tensor([[0.2961, 0.5166, 0.2517, 0.6886, 0.0740, 0.8665, 0.1366, 0.1025, 0.1841,
         0.7264, 0.3153, 0.6871, 0.0756, 0.1966, 0.3164, 0.4017],
        [0.1186, 0.8274, 0.3821, 0.6605, 0.8536, 0.5932, 0.6367, 0.9826, 0.2745,
         0.6584, 0.2775, 0.8573, 0.8993, 0.0390, 0.9268, 0.7388],
        [0.7179, 0.7058, 0.9156, 0.4340, 0.0772, 0.3565, 0.1479, 0.5331, 0.4066,
         0.2318, 0.4545, 0.9737, 0.4606, 0.5159, 0.4220, 0.5786],
        [0.9455, 0.8057, 0.6775, 0.6087, 0.6179, 0.6932, 0.4354, 0.0353, 0.1908,
         0.9268, 0.5299, 0.0950, 0.5789, 0.9131, 0.0275, 0.1634],
        [0.3009, 0.5201, 0.3834, 0.4451, 0.0126, 0.7341, 0.9389, 0.8056, 0.1459,
         0.0969, 0.7076, 0.5112, 0.7050, 0.0114, 0.4702, 0.8526],
        [0.7320, 0.5183, 0.5983, 0.4527, 0.2251, 0.3111, 0.1955, 0.9153, 0.7751,
         0.6749, 0.1166, 0.8858, 0.6568, 0.8459, 0.3033, 0.6060],
        [0.9882, 0.8363, 0.9010, 0.3950, 0.8809, 0.1084, 0.5432, 0.2185, 0.3834,
         0.3720

In [7]:
W_key

Parameter containing:
tensor([[0.0204, 0.8290, 0.1063, 0.2062, 0.5058, 0.6522, 0.7905, 0.4298, 0.2427,
         0.4570, 0.6638, 0.2187, 0.0657, 0.7387, 0.1691, 0.2186],
        [0.9148, 0.1705, 0.0943, 0.8800, 0.2614, 0.5325, 0.9981, 0.3005, 0.9657,
         0.8973, 0.8862, 0.6483, 0.2746, 0.8148, 0.1575, 0.2087],
        [0.2590, 0.7162, 0.5689, 0.8181, 0.8286, 0.5292, 0.7914, 0.1387, 0.0221,
         0.0927, 0.7759, 0.9598, 0.3617, 0.7766, 0.1427, 0.4906],
        [0.4970, 0.3552, 0.2576, 0.7346, 0.4564, 0.4009, 0.8474, 0.1203, 0.8265,
         0.9441, 0.1928, 0.0263, 0.5696, 0.1197, 0.7091, 0.1012],
        [0.1098, 0.6353, 0.3719, 0.0574, 0.6951, 0.6766, 0.5674, 0.8267, 0.2993,
         0.9564, 0.1189, 0.9508, 0.8715, 0.0552, 0.4556, 0.2310],
        [0.9920, 0.4791, 0.7945, 0.9323, 0.1144, 0.8039, 0.0651, 0.3650, 0.2984,
         0.0324, 0.0290, 0.0179, 0.1132, 0.2206, 0.3352, 0.7797],
        [0.4196, 0.0050, 0.1368, 0.8588, 0.0121, 0.2541, 0.0475, 0.7690, 0.8418,
         0.5438

In [8]:
W_value

Parameter containing:
tensor([[2.2383e-01, 3.0465e-01, 3.0185e-01, 7.7719e-01, 4.9632e-01, 3.8457e-01,
         9.4751e-02, 5.4099e-01, 8.0899e-01, 8.1570e-01, 5.4314e-01, 9.5954e-01,
         3.7638e-01, 8.8847e-01, 7.7945e-01, 9.4166e-01],
        [7.5758e-01, 4.9898e-02, 7.4476e-01, 1.3877e-01, 1.6512e-01, 1.4907e-01,
         2.6847e-01, 5.0905e-02, 9.2707e-01, 2.8936e-01, 8.2721e-01, 9.4828e-01,
         8.1707e-01, 8.7183e-01, 5.1264e-01, 8.6063e-03],
        [8.0527e-01, 7.8735e-02, 6.2932e-01, 2.9138e-01, 8.2026e-01, 8.3362e-01,
         4.7395e-01, 3.2585e-01, 8.8695e-01, 3.4264e-01, 1.1503e-01, 1.7675e-01,
         2.1455e-02, 8.6990e-01, 8.7559e-01, 3.7270e-01],
        [7.2059e-01, 7.8469e-01, 2.9878e-01, 5.8486e-01, 4.1490e-01, 2.5936e-01,
         1.8493e-01, 2.5396e-01, 4.6260e-01, 4.3994e-01, 1.2095e-01, 4.5656e-02,
         4.3196e-01, 6.9407e-01, 6.6612e-01, 1.4987e-01],
        [7.6967e-01, 1.5432e-01, 2.5701e-01, 9.0780e-01, 6.2522e-01, 6.8266e-01,
         1.8458e-

In [9]:
x_2 = embedded_sentence[1]
q_2 = W_q.matmul(x_2)
key_2 = W_key.matmul(x_2)
value_2 = W_value.matmul(x_2)


print(q_2.shape)
print(key_2.shape)
print(value_2.shape)

torch.Size([24])
torch.Size([24])
torch.Size([28])


In [10]:
keys = W_key.matmul(embedded_sentence.T).T
values = W_value.matmul(embedded_sentence.T).T
print("keys.shape:", keys.shape)
print("values.shape:", values.shape)

keys.shape: torch.Size([6, 24])
values.shape: torch.Size([6, 28])


In [11]:
omega_24 = q_2.dot(keys[4])
print(omega_24)

tensor(11.1466, grad_fn=<DotBackward0>)


In [12]:
omega_2 = q_2.matmul(keys.T)
print(omega_2)

tensor([ 8.5808, -7.6597,  3.2558,  1.0395, 11.1466, -0.4800],
       grad_fn=<SqueezeBackward4>)


In [13]:
import torch.nn.functional as F

attention_weights_2 = F.softmax(omega_2 / d_k**0.5, dim=0)
print(attention_weights_2)



tensor([0.2912, 0.0106, 0.0982, 0.0625, 0.4917, 0.0458],
       grad_fn=<SoftmaxBackward0>)


In [14]:
context_vector_2 = attention_weights_2.matmul(values)

print(context_vector_2.shape)
print(context_vector_2)

torch.Size([28])
tensor([-1.5993,  0.0156,  1.2670,  0.0032, -0.6460, -1.1407, -0.4908, -1.4632,
         0.4747,  1.1926,  0.4506, -0.7110,  0.0602,  0.7125, -0.1628, -2.0184,
         0.3838, -2.1188, -0.8136, -1.5694,  0.7934, -0.2911, -1.3640, -0.2366,
        -0.9564, -0.5265,  0.0624,  1.7084], grad_fn=<SqueezeBackward4>)


In [15]:
h = 3

multihead_W_q = torch.nn.Parameter(torch.rand(h, d_q, d))
multihead_W_key = torch.nn.Parameter(torch.rand(h, d_k, d))
multihead_W_value = torch.nn.Parameter(torch.rand(h, d_v, d))

In [16]:
multihead_W_q

Parameter containing:
tensor([[[0.4293, 0.9718, 0.5944,  ..., 0.4626, 0.8429, 0.5212],
         [0.4363, 0.1431, 0.1026,  ..., 0.2799, 0.5159, 0.2680],
         [0.4354, 0.1799, 0.1770,  ..., 0.1917, 0.3999, 0.7527],
         ...,
         [0.8646, 0.4360, 0.6114,  ..., 0.6239, 0.4606, 0.4535],
         [0.6290, 0.8649, 0.6046,  ..., 0.5090, 0.5695, 0.5944],
         [0.3154, 0.4451, 0.4446,  ..., 0.1518, 0.4902, 0.0430]],

        [[0.1220, 0.3788, 0.3001,  ..., 0.8186, 0.5012, 0.1457],
         [0.0977, 0.2001, 0.5758,  ..., 0.6639, 0.1982, 0.9953],
         [0.5071, 0.1769, 0.2574,  ..., 0.4956, 0.1194, 0.0554],
         ...,
         [0.0566, 0.6935, 0.5907,  ..., 0.9647, 0.3388, 0.0946],
         [0.2464, 0.4463, 0.2591,  ..., 0.6841, 0.0402, 0.6472],
         [0.7686, 0.9288, 0.6814,  ..., 0.2088, 0.9470, 0.8280]],

        [[0.8965, 0.4508, 0.3851,  ..., 0.9463, 0.4930, 0.9744],
         [0.9650, 0.5882, 0.7301,  ..., 0.7742, 0.1992, 0.2771],
         [0.4842, 0.3611, 0.8470,  .

In [17]:
multihead_W_key

Parameter containing:
tensor([[[9.7887e-01, 2.1895e-01, 3.6962e-02,  ..., 1.0530e-01,
          5.5615e-01, 9.3379e-01],
         [5.4246e-01, 2.0623e-01, 2.1699e-01,  ..., 9.9575e-01,
          4.9068e-01, 9.3889e-01],
         [7.8064e-01, 3.2142e-01, 4.0875e-01,  ..., 7.8456e-01,
          7.7953e-01, 7.5624e-01],
         ...,
         [2.5601e-01, 5.1916e-01, 7.5737e-01,  ..., 1.3676e-01,
          5.1134e-01, 7.4585e-01],
         [3.1543e-01, 5.1715e-01, 2.3261e-01,  ..., 8.8312e-01,
          1.6098e-01, 4.2674e-01],
         [2.2507e-01, 3.5796e-01, 8.7905e-01,  ..., 8.4578e-01,
          6.6905e-01, 1.3806e-01]],

        [[3.6154e-02, 6.0171e-01, 7.9425e-02,  ..., 7.3659e-01,
          1.5216e-01, 2.7247e-01],
         [7.3454e-01, 2.4636e-01, 2.8786e-01,  ..., 5.4318e-01,
          9.9408e-01, 1.8536e-01],
         [5.5266e-02, 3.7982e-01, 9.9005e-01,  ..., 6.8820e-01,
          1.6709e-01, 7.2117e-01],
         ...,
         [2.1322e-02, 8.1476e-01, 3.2534e-01,  ..., 3.142

In [18]:
multihead_W_value

Parameter containing:
tensor([[[0.8126, 0.7680, 0.0982,  ..., 0.5989, 0.0454, 0.2555],
         [0.1407, 0.0555, 0.8892,  ..., 0.4315, 0.9336, 0.3189],
         [0.4381, 0.4662, 0.1572,  ..., 0.6475, 0.3617, 0.7963],
         ...,
         [0.7508, 0.7382, 0.2218,  ..., 0.2933, 0.9148, 0.6330],
         [0.9484, 0.3823, 0.1131,  ..., 0.1616, 0.8981, 0.8360],
         [0.3942, 0.3983, 0.3108,  ..., 0.8498, 0.3343, 0.3563]],

        [[0.2753, 0.2350, 0.8561,  ..., 0.4399, 0.7952, 0.3370],
         [0.5276, 0.9645, 0.1278,  ..., 0.2874, 0.0529, 0.4392],
         [0.2921, 0.2099, 0.6127,  ..., 0.8684, 0.8080, 0.6249],
         ...,
         [0.1078, 0.9959, 0.5275,  ..., 0.2236, 0.0915, 0.9276],
         [0.4429, 0.9930, 0.8635,  ..., 0.4881, 0.8600, 0.1000],
         [0.9975, 0.7570, 0.9336,  ..., 0.9869, 0.9412, 0.7387]],

        [[0.5746, 0.4230, 0.9776,  ..., 0.2349, 0.3183, 0.4343],
         [0.0319, 0.8467, 0.4079,  ..., 0.6107, 0.7830, 0.7642],
         [0.9717, 0.6954, 0.2992,  .

In [19]:
multihead_q_2 = multihead_W_q.matmul(x_2)
print(multihead_q_2.shape)

torch.Size([3, 24])


In [20]:
multihead_key_2 = multihead_W_key.matmul(x_2)
multihead_value_2 = multihead_W_value.matmul(x_2)

In [21]:
multihead_key_2.shape

torch.Size([3, 24])

In [22]:
multihead_value_2.shape

torch.Size([3, 28])

In [23]:
stacked_inputs = embedded_sentence.T.repeat(3,1,1)
print(stacked_inputs.shape)

torch.Size([3, 16, 6])


In [24]:
multihead_keys = torch.bmm(multihead_W_key, stacked_inputs)
multihead_values = torch.bmm(multihead_W_value, stacked_inputs)
print("multihead_keys.shape:", multihead_keys.shape)
print("multihead_values.shape:", multihead_values.shape)

multihead_keys.shape: torch.Size([3, 24, 6])
multihead_values.shape: torch.Size([3, 28, 6])


In [25]:
multihead_keys.shape

torch.Size([3, 24, 6])

In [26]:
multihead_values.shape

torch.Size([3, 28, 6])

In [27]:
multihead_keys = multihead_keys.permute(0,2,1)
multihead_values = multihead_values.permute(0, 2, 1)
print("multihead_keys.shape:", multihead_keys.shape)
print("multihead_values.shape:",multihead_values.shape)

multihead_keys.shape: torch.Size([3, 6, 24])
multihead_values.shape: torch.Size([3, 6, 28])


In [28]:
torch.manual_seed(123)
d = embedded_sentence.shape[1]
print("embedded_entence.shape:", embedded_sentence.shape)
d_q, d_k, d_v = 24,24,8
W_q = torch.rand(d_q, d)
W_key = torch.rand(d_k, d)
W_value = torch.rand(d_v, d)
x_2 = embedded_sentence[1]
q_2 = W_q.matmul(x_2)
print("q.shape", q_2.shape)
keys = W_key.matmul(embedded_sentence.T).T
values = W_value.matmul(embedded_sentence.T).T

print("keys.shape:", keys.shape)
print("values.shape:", values.shape)

embedded_entence.shape: torch.Size([6, 16])
q.shape torch.Size([24])
keys.shape: torch.Size([6, 24])
values.shape: torch.Size([6, 8])


In [29]:
embedded_sentence_2 = torch.rand(8,16)

keys = W_key.matmul(embedded_sentence_2.T).T
values = W_value.matmul(embedded_sentence_2.T).T

print("keys.shape:", keys.shape)
print("values.shape:", values.shape)

keys.shape: torch.Size([8, 24])
values.shape: torch.Size([8, 8])
